In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[-0.71764706 -0.59215686 -0.58431373 ... -0.70980392 -0.61568627
  -0.56078431]
 [-0.34117647 -0.29411765 -0.28627451 ... -0.71764706 -0.74901961
  -0.77254902]
 [-0.42745098 -0.12156863  0.69411765 ...  0.41176471  0.41176471
   0.37254902]
 ...
 [ 1.          0.98431373  0.97647059 ...  0.43529412  0.41176471
   0.41960784]
 [-0.69411765 -0.69411765 -0.68627451 ...  0.43529412  0.41176471
   0.40392157]
 [ 0.81960784  0.81176471  0.81176471 ... -0.38823529 -0.38039216
  -0.39607843]] [3 8 5 7 6 9 2 7 0 1]


In [3]:
def inception_block(x, output_channel_for_each_path, name):
    """inception block implementation"""
    """
    Args:
    - x:
    - output_channel_for_each_path: eg:[10, 20, 5]
    - name:
    """
    with tf.variable_scope(name):
        conv1_1 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[0],
                                   (1,1),
                                  strides=(1,1),
                                  padding='same',
                                  activation=tf.nn.relu,
                                  name='conv1_1')
        conv3_3 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[1],
                                   (3,3),
                                  strides=(1,1),
                                  padding='same',
                                  activation=tf.nn.relu,
                                  name='conv3_3')
        conv5_5 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[2],
                                   (5,5),
                                  strides=(1,1),
                                  padding='same',
                                  activation=tf.nn.relu,
                                  name='conv5_5')
        max_pooling =tf.layers.max_pooling2d(x, 
                                            (2,2),
                                            (2,2),
                                            name='max_pooling')
        
    max_pooling_shape = max_pooling.get_shape().as_list()[1:]
    input_shape = x.get_shape().as_list()[1:]
    width_padding = (input_shape[0] - max_pooling_shape[0]) // 2
    height_padding = (input_shape[1] - max_pooling_shape[1]) // 2
    padded_pooling =tf.pad(max_pooling,
                          [[0,0],
                          [width_padding, width_padding],
                          [height_padding, height_padding],
                          [0, 0]])
    concat_layer = tf.concat(
        [conv1_1, conv3_3, conv5_5, padded_pooling],
        axis=3 )
    return concat_layer

In [4]:
x = tf.placeholder(tf.float32, [None, 3072])

# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# inception-net思想 acc: 0.73500
# conv1: 神经元图， feature_map, 输出图像
conv1 = tf.layers.conv2d(x_image,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1')

pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

inception_2a = inception_block(pooling1,
                               [16, 16, 16],
                               name = 'inception_2a')
inception_2b = inception_block(inception_2a,
                               [16, 16, 16],
                               name = 'inception_2b')

pooling2 = tf.layers.max_pooling2d(inception_2b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

inception_3a = inception_block(pooling2,
                               [16, 16, 16],
                               name = 'inception_3a')
inception_3b = inception_block(inception_3a,
                               [16, 16, 16],
                               name = 'inception_3b')

pooling3 = tf.layers.max_pooling2d(inception_3b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')

flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0714 20:07:49.035041 15972 deprecation.py:323] From <ipython-input-4-8d596c44b800>:17: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0714 20:07:49.311229 15972 deprecation.py:506] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0714 20:07:51.056886 15972 deprecation.py:323] From <ipython-input-4-8d596c44b800>:22: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0714 20:07:51.690627 15972 deprecation.py:323] From <ipython-input-4-8d596c44

In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 500 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 2.34802, acc:0.10000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.09200
[Train] step: 100, loss: 2.03153, acc:0.35000
[Train] step: 200, loss: 1.64638, acc:0.45000
[Train] step: 300, loss: 1.65541, acc:0.45000
[Train] step: 400, loss: 1.44934, acc:0.55000
[Train] step: 500, loss: 1.67185, acc:0.45000
(10000, 3072) (10000,)
[Test] Step: 501, acc: 0.48350
[Train] step: 600, loss: 1.76905, acc:0.40000
[Train] step: 700, loss: 1.22932, acc:0.60000
[Train] step: 800, loss: 1.27745, acc:0.65000
[Train] step: 900, loss: 0.97934, acc:0.70000
[Train] step: 1000, loss: 1.25801, acc:0.60000
(10000, 3072) (10000,)
[Test] Step: 1001, acc: 0.57600
[Train] step: 1100, loss: 1.16453, acc:0.65000
[Train] step: 1200, loss: 1.44902, acc:0.45000
[Train] step: 1300, loss: 1.15622, acc:0.50000
[Train] step: 1400, loss: 0.79602, acc:0.80000
[Train] step: 1500, loss: 1.16650, acc:0.60000
(10000, 3072) (10000,)
[Test] Step: 1501, acc: 0.59000
[Train] step: 1600, loss: 1.23606, acc:0.600

In [6]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Test] Step: 10000, acc: 0.73500
